# Save and load models
https://www.tensorflow.org/tutorials/keras/save_and_load

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
tf.version.VERSION

'2.3.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training


Epoch 1/10
 1/32 [..............................] - ETA: 0s - loss: 2.3937 - sparse_categorical_accuracy: 0.0312
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 1.1513 - sparse_categorical_accuracy: 0.6680 - val_loss: 0.7182 - val_sparse_categorical_accuracy: 0.7800
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.7015 - sparse_categorical_accuracy: 0.7500
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.4336 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5294 - val_sparse_categorical_accuracy: 0.8410
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.1161 - sparse_categorical_accuracy: 1.0000
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2878 - sparse_categorical_accuracy: 0.9230 - val_loss: 0.4610 - val_sparse_categorical_accuracy: 0.8520
Epoch 4/10
 1/32

In [5]:
# ls {checkpoint_dir}

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3973 - sparse_categorical_accuracy: 0.0660
Untrained model, accuracy:  6.60%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4464 - sparse_categorical_accuracy: 0.8560
Restored model, accuracy: 85.60%


In [8]:
# checkpoint callback

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [9]:
# ls {checkpoint_dir}

In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4761 - sparse_categorical_accuracy: 0.8840
Restored model, accuracy: 88.40%


In [12]:
# Manually save weights

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4761 - sparse_categorical_accuracy: 0.8840
Restored model, accuracy: 88.40%


In [13]:
# Save the entire model

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1231 - sparse_categorical_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4301 - sparse_categorical_accuracy: 0.8640
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2781 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2063 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1500 - sparse_categorical_accuracy: 0.9720
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [14]:
# my_model directory
# !ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
# !ls saved_model/my_model

In [15]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4186 - sparse_categorical_accuracy: 0.8660
Restored model, accuracy: 86.60%
(1000, 10)


In [17]:
# HDF5 format

# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1889 - sparse_categorical_accuracy: 0.6650
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4145 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2839 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1980 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1493 - sparse_categorical_accuracy: 0.9670


In [18]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4408 - sparse_categorical_accuracy: 0.8600
Restored model, accuracy: 86.00%
